# How to generate reports in plain text or markdown format

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * `report` action generate plain text output
  

### Action `report`

Action `report` writes some content to an output stream. The input can either be a string or content of one or more files specified by option `input`. The output is determined by parameter `output`, and command line option `-r`.

* If `output='filename'`, the content will be written to a file.
* If `output=obj` and `obj` has a `write` function (e.g. a file handle), the content will be passed to the `write` function
* If output is unspecified and no filename is specified from option `-r`, the content will be written to standard output.
* If output is unspecified and a filename is specified with option `-r`, the content will be appended to specified file.

For example, the content of `report` actions is printed to standard output if no output is specified.

In [34]:
%run

[10]
report: expand=True
    Runing {step_name}

[20]
report: expand=True
    Runing {step_name}


Runing default_10

Runing default_20



We can specify an output file with option `output`, but the output will be overwritten if multiple actions write to the same file

In [35]:
%sandbox

%preview report.txt
%run
[10]
report: output='report.txt', expand=True
    Runing {step_name}

[20]
report: output='report.txt', expand=True
    Runing {step_name}

Action `report` can also take the content of one or more input files and write them to the output stream, after the script content (if specified). For example, the `report` action in the following example writes the content of `out.txt` to the default report stream (which is the standard output in this case).

In [36]:
%sandbox
%run

[10]
output: 'out.txt'
run:
   # run some command and generate a out.txt
   echo "* some result " > out.txt

[20]
report: input='out.txt'
Summary Report:

# run some command and generate a out.txt
echo "* some result " > out.txt

Summary Report:

* some result



## A summarization step

You can write a report at the end of the workflow that summarizes the results of previous steps. For example, in the following example, the `report` action summarizes previous steps and writes a report to the standard output.

In [1]:
%sandbox
%run

[10]
output: 'a.jpg'
R: expand=True
    jpeg({_output:r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

[20]
output: 'a.txt'
run:
    echo "100" > a.txt

[100]
input: 'a.jpg', 'a.txt'
with open('a.txt') as a:
    res = a.read()

report: expand=True
    * Figure
    ![figure](a.jpg)
    * result
    {res}
   

echo "100" > a.txt

* Figure
![figure](a.jpg)
* result
100



It is a pretty bad idea to write report to standard output because other actions can also write to it. You should therefore use the `output` option of the `report` action to specify an output file of the `report` action.

In [2]:
%sandbox
%preview summary.md
%run

[10]
output: 'a.jpg'
R: expand=True
    jpeg({_output:r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

[20]
output: 'a.txt'
run:
    echo "100" > a.txt

[counts: shared='counts']
input: 'a.txt'
with open('a.txt') as a:
    counts = a.read()

[100]
input: 'a.jpg'
depends: sos_variable('counts')

report: output='summary.md', expand=True
    * Figure
    ![figure](a.jpg)
    * result
    {counts}
   

echo "100" > a.txt



Instead of outputting reports in `.md` format and rendering them outside of SoS, you can also render them inside SoS using action `pandoc` or `Rmarkdown`. For example,

In [3]:
%sandbox
%preview summary.html
[10]
output: 'a.jpg'
R: expand=True
    jpeg({_output:r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

[20]
output: 'a.txt'
run:
    echo "100" > a.txt

[counts: shared='counts']
input: 'a.txt'
with open('a.txt') as a:
    counts = a.read()

[100]
input: 'a.jpg'
depends: sos_variable('counts')

pandoc: output='summary.html', expand=True
    * Figure
    ![figure](a.jpg)
    * result
    {counts}

## Reporting to multiple output files

If you have a large workflow with several steps or modules, it makes sense to write several reports and join them at last. For example, in the following example, an output file is specified for each `report` and a summary step is used at the end to collect and process them.

In [4]:
%sandbox --dir ~/tmp
%preview summary.html
[10]
output: 'a.jpg'
R: expand=True
    jpeg({_output:r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

report: output='figure.md', expand=True
    * Figure
    ![figure]({output})    

[20]
output: 'a.txt'
counts = 100
run: expand=True
    echo "{counts}" > a.txt

report: output='result.md', expand=True
    * result
    {counts}

[100]
pandoc: input=['figure.md', 'result.md'], output='summary.html'
    Final report

You will notice that both `script` and `input` are specified to action `pandoc`. In this case the `script` is put before the content of each `input` files, making it a perfect place to write headers and summaries.

## Further reading

* 